In [1]:
import glob
import json
import os

import pandas as pd

In [2]:
folder_mapping = {
    "1": "计算机体系结构/并行与分布计算/存储系统",
    "2": "计算机网络",
    "3": "网络与信息安全",
    "4": "软件工程/系统软件/程序设计语言",
    "5": "数据库/数据挖掘/内容检索",
    "6": "计算机科学理论",
    "7": "计算机图形学与多媒体",
    "8": "人工智能",
    "9": "人机交互与普适计算",
    "10": "交叉/综合/新兴",
}

<h2>生成 PaperInfoList.json 文件，该文件包含所有文件夹的嵌套结构，每个文件夹下包含所有 JSON 文件的名称和内容。</h2>

In [13]:
# 文件夹路径模板，文件夹命名为1到10
base_folder = './paper_db'
folders = [os.path.join(base_folder, str(i)) for i in range(1, 11)]  # 文件夹命名为1到10

# 用来存储最终的嵌套结构
result = []

# 遍历每个文件夹
for folder in folders:
    # 获取当前文件夹中所有 JSON 文件路径
    json_files = glob.glob(os.path.join(folder, '*.json'))

    # 提取文件名（去掉路径和后缀）
    file_names = [os.path.splitext(os.path.basename(file))[0] for file in json_files]

    # 获取文件夹的名称（从映射字典中获取）
    folder_name = os.path.basename(folder)
    folder_label = folder_mapping.get(folder_name, folder_name)  # 如果没有映射，使用文件夹名本身

    # 构建嵌套结构
    folder_structure = {
        'value': folder_label,  # 映射后的文件夹名称作为 value
        'label': folder_label,  # 映射后的文件夹名称作为 label
        'children': []  # 存储每个文件夹下的文件名
    }
    for file in json_files:
        # 获取文件名（去掉路径和扩展名）
        file_name = os.path.splitext(os.path.basename(file))[0]
        # 读取 JSON 文件内容
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for item in data:
                if 'venues' in item and len(item['venues']) > 0:
                    # 有 venues 字段，说明是则把文件名作为 value，label 作为 label，构建一个新的嵌套结构，一个文件只加入一次
                    folder_structure['children'].append({
                        'value': file_name,  # 文件名作为 value
                        'label': file_name  # 文件名作为 label
                    })
                    break
    # 将该文件夹的结构添加到结果列表
    result.append(folder_structure)

# 将结果保存为 JSON 文件
output_file = 'data/PaperInfoList.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

<h2>生成 PaperTotalList.json 文件，该文件包含所有文件夹的嵌套结构，每个文件夹下包含所有 JSON 文件的名称和内容。</h2>

In [4]:
# 假设文件夹的命名为1到10
base_folder = './paper_db'
folders = [os.path.join(base_folder, str(i)) for i in range(1, 11)]  # 文件夹命名为1到10

# 用来存储所有文件夹的数据
all_folders_data = {}

# 遍历每个文件夹
for folder_id, folder in enumerate(folders, 1):  # 1到10编号文件夹
    # 获取当前文件夹中所有 JSON 文件路径
    json_files = glob.glob(os.path.join(folder, '*.json'))

    # 当前文件夹的数据
    folder_data = []

    # 逐个读取 JSON 文件
    for file in json_files:
        # 获取文件名（去掉路径和扩展名）
        file_name = os.path.splitext(os.path.basename(file))[0]
        # 读取 JSON 文件内容
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            for item in data:
                if 'venues' in item:
                    # 遍历 'venues' 下的每个元素
                    for venue in item['venues']:
                        if 'papers' in venue:
                            # 将 papers 转化为新的格式
                            venue['papers'] = [{"PaperName": paper} for paper in venue['papers']]
                elif 'papers' in item:
                    # 处理没有 venues 字段的情况
                    item['venues'] = [{
                        "sub_name_abbr": "未知",
                        "sub_name": "未知",
                        "count": item['count'],
                        "papers": [{"PaperName": paper} for paper in item['papers']]
                    }]
                    del item['papers'], item['count']

            # 将文件名和文件内容作为字典添加到当前文件夹的数据中
            folder_data.append({
                "paperName": file_name,
                "data": data
            })

    # 使用映射字典的中文名称为当前文件夹命名
    folder_name = folder_mapping[str(folder_id)]
    all_folders_data[folder_name] = folder_data

# # 将所有数据保存为 JSON 格式
output_file = './data/PaperTotalList.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_folders_data, f, ensure_ascii=False, indent=4)

print(f"保存成功，文件保存在 {output_file}")


保存成功，文件保存在 ./data/PaperTotalList.json


In [15]:
# 读取 JSON 文件
with open('./data/PaperTotalList.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 展平嵌套结构并转换为 DataFrame
flattened_data = []

# 遍历每个文件夹（中文名称）
for folder_name, papers in data.items():
    # 遍历每个文件（即paperName和data）
    for paper in papers:
        paper_name = paper['paperName']
        paper_data = paper['data']

        # 如果需要更复杂的结构化，可以在这里修改，譬如提取某些字段
        for item in paper_data:
            # 将 paper_name 和对应的 JSON 数据合并到一起
            flattened_data.append({
                'folderName': folder_name,  # 文件夹的中文名称
                'paperName': paper_name,  # 文件名（paperName）
                'data': item  # JSON 数据的具体内容
            })

# 转换为 DataFrame
df = pd.DataFrame(flattened_data)

<h2>生成词云图</h2>

In [5]:
mapping = {
    1: "计算机体系结构并行与分布计算存储系统",
    2: "计算机网络",
    3: "网络与信息安全",
    4: "软件工程系统软件程序设计语言",
    5: "数据库数据挖掘内容检索",
    6: "计算机科学理论",
    7: "计算机图形学与多媒体",
    8: "人工智能",
    9: "人机交互与普适计算",
    10: "交叉综合新兴",
}

In [15]:
#  导入库
import os
from wordcloud import WordCloud


# 创建词云
def handle_wordcloud(category):
    folder_path = f'./paper_db/{category}'  # 论文数据所在的文件夹路径
    papers_text = handle_data(folder_path)  # 获取论文标题文本
    # 加载图片
    # 生成词云
    wc = WordCloud(background_color='white',  # 背景色为白色，适合学术主题
                   colormap='Blues',  # 使用蓝色调，通常与学术风格契合
                   max_words=1500,  # 显示最多2000个词
                   max_font_size=100,  # 最大字体大小为100
                   random_state=42,  # 随机种子，保证每次生成相同的结果
                   width=900,  # 更宽的图像，以便容纳更多的词
                   height=850,  # 合理的高度
                   contour_color='black',  # 设置边框颜色为黑色，增加视觉对比
                   contour_width=1,  # 细边框，使词云看起来更加精致
                   prefer_horizontal=0.7,  # 控制水平文字的比例，稍微偏向垂直，以适应学术格式
                   min_font_size=10,  # 设置最小字体大小，避免字体过小
                   normalize_plurals=True,  # 合并单数和复数形式的词
                   stopwords=None,  # 根据需要添加自定义停用词
                   scale=3,  # 提升图像质量，适合打印或展示
                   )
    wc.generate(papers_text)
    # 保存词云图片
    wc.to_file('../assets/' + f"{mapping[category]}.png")


# 数据处理函数
def handle_data(folder_path):
    papers_list = []
    # 遍历文件夹中的所有文件
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):  # 只处理 JSON 文件
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)  # 加载 JSON 数据
                # 遍历每个条目，提取论文标题
                for entry in data:
                    if 'venues' in entry:
                        for venue in entry['venues']:
                            if 'papers' in venue:
                                papers_list.extend(venue['papers'])
    papers_text = " ".join(papers_list)  # 将论文标题合并成文本
    return papers_text


In [17]:
# 更具文件顺序生成词云
for category in range(1, 11):
    handle_wordcloud(category)

<h2>修改和映射会议名称</h2>

In [19]:
ccf_data = pd.read_csv('./paper_db/ccf_catalog.csv')

In [20]:
ccf_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         612 non-null    int64 
 1   abbr       612 non-null    object
 2   name       612 non-null    object
 3   publisher  610 non-null    object
 4   url        612 non-null    object
 5   rank       612 non-null    object
 6   type       612 non-null    object
 7   category   612 non-null    object
dtypes: int64(1), object(7)
memory usage: 38.4+ KB


In [18]:
import pandas as pd


# 定义填充函数
def fill_abbr(row):
    if pd.isna(row['abbr']):
        return ''.join([word[0].upper() for word in row['name'].split() if word])  # 获取大写字母并合并
    return row['abbr']


# 填充缺失的 abbr
ccf_data['abbr'] = ccf_data.apply(fill_abbr, axis=1)

ccf_data.to_csv('./paper_db/ccf_catalog.csv', index=False)

<h2> 生成知识图谱</h2>

In [ ]:
folder_mapping = {
    "计算机体系结构/并行与分布计算/存储系统": "1",
    "计算机网络": "2",
    "网络与信息安全": "3",
    "软件工程/系统软件/程序设计语言": "4",
    "数据库/数据挖掘/内容检索": "5",
    "计算机科学理论": "6",
    "计算机图形学与多媒体": "7",
    "人工智能": "8",
    "人机交互与普适计算": "9",
    "交叉/综合/新兴": "10",
}

In [21]:
import json

# 使用 json 库读取文件
with open('./data/PaperTotalList.json', 'r', encoding='utf-8') as f:
    PaperTotalList = json.load(f)

# 用于存储转换后的数据
output_data = {
    "nodes": [],
    "edges": []
}

# 用于跟踪ID的映射
node_id_map = {}
node_id_counter = 0

# 遍历输入数据并构建新的结构
for domain in PaperTotalList:
    output_data['nodes'].append({
        "id": domain,
        "data": {
            "count": 10000,
            "cluster": node_id_counter
        }
    })
    node_id_counter += 1
    for paper in PaperTotalList[domain]: # 辨别每个领域
        paper_name = paper['paperName']
        paper_count = 0
        for item in paper['data']:  # 遍历每个论文
            if 'venues' in item:  # 处理有 venues 字段的情况
                for venue in item['venues']: # 遍历 venues 字段
                    if 'count' in venue:
                        paper_count += venue['count']
            elif 'count' in item:  # 处理没有 venues 字段的情况
                paper_count += item['count']
        output_data['nodes'].append({
            "id": paper_name,
            "data": {
                "count": paper_count,
                "cluster": domain
            }
        })
        output_data['edges'].append({
            "source": paper_name,
            "target": domain
        })


# print(output_data['nodes'])
# 保存为json、
with open('KnowledgeGraph.json', 'w', encoding='utf-8') as f:
    json.dump(output_data, f, ensure_ascii=False, indent=4)
